In [1]:
import os
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [5]:
from pydeseq2.utils import build_design_matrix
from deconveil.dds import deconveil_fit
from deconveil.inference import Inference
from deconveil.default_inference import DefInference
from deconveil.utils_CNaware import *
from deconveil import deconveil_fit
from deconveil.ds import deconveil_stats

In [13]:
def run_pydeseq2(rna_counts, metadata, output_path, design_factors="condition", alpha=0.05):
    """
    Runs PyDESeq2 analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DESeq2 analysis
    inference = DefaultInference(n_cpus=8)
    dds = DeseqDataSet(
        counts=rna_counts,
        metadata=metadata,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
    )

    # Fit DESeq2 model
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()
    
    if dds.refit_cooks:
        dds.refit()

    # Perform statistical analysis
    stat_res_pydeseq = DeseqStats(dds, alpha=alpha, cooks_filter=True, independent_filter=True)
    stat_res_pydeseq.run_wald_test()

    if stat_res_pydeseq.cooks_filter:
        stat_res_pydeseq._cooks_filtering()
    stat_res_pydeseq.p_values

    if stat_res_pydeseq.independent_filter:
        stat_res_pydeseq._independent_filtering()
    else:
        stat_res_pydeseq._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_pydeseq.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_pydeseq.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNnaive_{n_samples}_{n_genes}.csv")
    stat_res_pydeseq.results_df.to_csv(results_path)
    return(stat_res_pydeseq.results_df)


def run_deconveil(rna_counts, metadata, cnv, output_path, design_factors="condition", alpha=0.05):
    """
    Runs DeConveil analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        cnv (pd.DataFrame): Copy number variation (CNV) data matrix  with genes as rows and samples as columns.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DeConveil inference
    inference = DefInference(n_cpus=8)

    # Fit DeConveil model
    dds = deconveil_fit(
        counts=rna_counts,
        metadata=metadata,
        cnv=cnv,
        design_factors=design_factors,
        inference=inference,
        refit_cooks=True
    )
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()

    if dds.refit_cooks:
        dds.refit()  # Replace outlier counts

    # Statistical analysis
    stat_res_deconveil = deconveil_stats(
        dds, 
        alpha=alpha, 
        independent_filter=True, 
        cooks_filter=True
    )
    stat_res_deconveil.run_wald_test()

    if stat_res_deconveil.independent_filter:
        stat_res_deconveil._independent_filtering()
    else:
        stat_res_deconveil._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_deconveil.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_deconveil.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNaware_{n_samples}_{n_genes}.csv")
    stat_res_deconveil.results_df.to_csv(results_path)
    return(stat_res_deconveil.results_df)

In [9]:
def process_and_save_single_run(n_samples, n_genes, noise_level):
    try:
        noise_label = f"noise_{int(noise_level * 100)}" if noise_level > 0 else "no_noise"
        base_path = f"/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/data/simulations_4/{noise_label}/"

        # Updated filenames to match the actual format
        rna_path = os.path.join(base_path, f"{n_samples}_{n_genes}_rna.csv")
        cnv_path = os.path.join(base_path, f"{n_samples}_{n_genes}_cn.csv")
        metadata_path = os.path.join(base_path, f"{n_samples}_{n_genes}_metadata.csv")

        # Check if files exist before loading them
        if not os.path.exists(rna_path):
            raise FileNotFoundError(f"RNA file not found: {rna_path}")
        if not os.path.exists(cnv_path):
            raise FileNotFoundError(f"CNV file not found: {cnv_path}")
        if not os.path.exists(metadata_path):
            raise FileNotFoundError(f"Metadata file not found: {metadata_path}")

        # Load data
        rna = pd.read_csv(rna_path, index_col=0).T
        cnv = pd.read_csv(cnv_path, index_col=0).T
        #cnv = (cnv * 2).astype(int)
        metadata = pd.read_csv(metadata_path, index_col=0)

        # Output directories
        output_base = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_4"
        pydeseq_output_path = os.path.join(output_base, f"{noise_label}/single_run_pydeseq/")
        deconveil_output_path = os.path.join(output_base, f"{noise_label}/single_run_deconveil/")
        os.makedirs(pydeseq_output_path, exist_ok=True)
        os.makedirs(deconveil_output_path, exist_ok=True)

        # Run tools
        res_pydeseq = run_pydeseq2(rna, metadata, pydeseq_output_path)
        res_deconveil = run_deconveil(rna, metadata, cnv, deconveil_output_path)

        print(f"✅ Processed single run | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Error in single run | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}: {e}")


In [15]:
#sample_sizes = [10, 20, 40, 100]
#gene_counts = [3000]
#noise_levels = [0, 0.05, 0.10, 0.15, 0.20]

#for n_samples in sample_sizes:
    #for n_genes in gene_counts:
        #for noise in noise_levels:
            #process_and_save_single_run(n_samples, n_genes, noise)

In [23]:
sample_sizes = [10,20,40,60]
gene_counts = [5000]
num_replicates = 5
#noise_levels = [0, 0.05, 0.10, 0.15, 0.20]
noise_levels = [0.05]

In [25]:
def process_and_save_replicates(replicate_num, n_samples, n_genes, noise_level):
    try:
        noise_label = f"noise_{int(noise_level * 100)}" if noise_level > 0 else "no_noise"
        base_path = f"/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/data/simulations_4/{noise_label}/"

        # Updated filenames to match your actual format
        rna_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_rna.csv")
        cnv_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_cn.csv")
        metadata_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_metadata.csv")

        # Load data
        rna = pd.read_csv(rna_path, index_col=0).T
        cnv = pd.read_csv(cnv_path, index_col=0).T
        cnv = (cnv * 2).astype(int)
        metadata = pd.read_csv(metadata_path, index_col=0)

        # Output directories
        output_base = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_4"
        pydeseq_output_path = os.path.join(output_base, f"{noise_label}/replicates_pydeseq/")
        deconveil_output_path = os.path.join(output_base, f"{noise_label}/replicates_deconveil/")
        os.makedirs(pydeseq_output_path, exist_ok=True)
        os.makedirs(deconveil_output_path, exist_ok=True)

        # Run tools
        res_pydeseq = run_pydeseq2(rna, metadata, pydeseq_output_path)
        res_deconveil = run_deconveil(rna, metadata, cnv, deconveil_output_path)

        print(f"✅ Processed replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}")
    except Exception as e:
        print(f"Error in replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}: {e}")

In [27]:
for n_samples in sample_sizes:
    for n_genes in gene_counts:
        for noise in noise_levels:
            for replicate_num in range(1, num_replicates + 1):
                process_and_save_replicates(replicate_num, n_samples, n_genes, noise)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.92 seconds.

Fitting dispersion trend curve...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 1.38 seconds.

Fitting LFCs...
... done in 0.50 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 147 outlier genes.

Fitting dispersions...
... done in 0.06 seconds.

Fitting MAP dispersions...
... done in 0.06 seconds.

Fitting LFCs...
... done in 0.06 seconds.

Running Wald tests...
... done in 0.16 seconds.

Fitting MAP LFCs...
... done in 0.89 seconds.



Log2 fold change & Wald test p-value: condition B vs A
                baseMean  log2FoldChange     lfcSE      stat    pvalue  \
PCDHGA10      286.524953        0.389284  0.376817  1.328283  0.184085   
RPP25         607.495120        1.064420  0.359130  3.414048  0.000640   
KRT17       21157.817270       -0.131355  0.604678 -1.081166  0.279623   
FRMD4A       1932.820656       -0.817331  0.220808 -3.970332  0.000072   
AC091178.2     20.364374       -0.251086  0.358896 -0.895297  0.370629   
...                  ...             ...       ...       ...       ...   
AL137026.3     13.479077       -1.784575  0.577246 -3.685254  0.000228   
AL138756.1    198.372976       -0.057470  0.146361 -0.412118  0.680253   
AMY2B         312.171153       -1.433960  0.369896 -4.307787  0.000016   
MPP3          386.737679        0.274161  0.310631  1.060257  0.289028   
USH2A          15.007107       -0.517032  0.557254 -1.490819  0.136009   

                padj  
PCDHGA10    0.304978  
RPP25     

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.81 seconds.

Fitting dispersion trend curve...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 1.09 seconds.

Fitting LFCs...
... done in 0.56 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 258 outlier genes.

Fitting dispersions...
... done in 0.04 seconds.

Fitting MAP dispersions...


replace_mask before filtering: (20, 258)
Number of True values in replace_mask: 280
replacement_counts_trimmed shape: (19, 258)


... done in 0.06 seconds.

Fitting LFCs...
... done in 0.05 seconds.

Running Wald tests...
... done in 0.21 seconds.

Fitting MAP LFCs...
... done in 0.86 seconds.



...

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.26 seconds.

Fitting dispersion trend curve...
... done in 0.09 seconds.

Fitting MAP dispersions...
... done in 1.19 seconds.

Fitting LFCs...
... done in 0.64 seconds.

Calculating cook's distance...
... done in 0.07 seconds.

Replacing 1058 outlier genes.



replace_mask before filtering: (120, 1058)
Number of True values in replace_mask: 1312
replacement_counts_trimmed shape: (86, 1058)


Fitting dispersions...
... done in 0.15 seconds.

Fitting MAP dispersions...
... done in 0.34 seconds.

Fitting LFCs...
... done in 0.10 seconds.

Running Wald tests...
... done in 0.29 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: condition B vs A
               baseMean  log2FoldChange     lfcSE       stat        pvalue  \
LINC01956     13.501620        2.108937  0.186922  -2.866898  4.145160e-03   
ETNPPL        17.203005       -0.003082  0.321477  -0.006273  9.949949e-01   
STAMBP      2962.147451       -0.017745  0.054949  -0.462984  6.433758e-01   
FCGR2C       221.932137       -1.040274  0.202702  -5.281874  1.278693e-07   
ZNF835       136.386502       -0.601705  0.191981  -3.534085  4.091899e-04   
...                 ...             ...       ...        ...           ...   
CERT1       4057.256191       -0.132665  0.061267   0.101440  9.192014e-01   
ZNF703      3980.124921        2.079595  0.066740   1.741147  8.165789e-02   
PPM1K       1563.933069       -0.681399  0.163116  -4.196770  2.707482e-05   
CENPA        260.550292        2.832171  0.226039  12.596564  2.205459e-36   
AC008731.1    11.338384       -0.660508  0.197061  -3.119849  1.809441e-03   

        

... done in 1.01 seconds.

